 Albina Chowdury, Cesar Cisneros, Estelle Hooper, Eva Ruse

# Dataset Creation & Cleaning
For our final project, we spent a substantial amount of time creating our datasets `spotify2019` and
`spotify2020`. First, we web scraped 40 of the [official Spotify top 200 chart webpages](https://spotifycharts.com/regional). Second, we used the Spotify API to gain Spotify-generated audio features of the songs we scraped from the charts. These audio features are numeric scores that rate a song's feature, such as danceability, valence, or loudness. 

This notebook documents our process of generating these datasets, explaining the code we wrote and the setbacks we faced.
<br><br>
Table of Contents
<br>
I. [Introduction: Why did we decide to make our own dataset?](#introduction)
<br>
II. [Cleaning Happiness Score Datasets](#cleaning)
<br>
III. [Web Scraping Spotify Charts](#scraping)
<br>
IV. [Spotify API](#api)
<br>

### I. Introduction: Why did we decide to make our own dataset?<a name="introduction"></a>
#### Research Question: Is there a relationship between how happy a country is and the music the people of this country listen to?
When we first asked our main research question at the start of the semester we had planned on using this dataset on Kaggle: [Spotify's Worldwide Daily Song Ranking (2017)](https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking).
It contains the top 200 streamed songs per day each for 54 countries (almost 4,000,000 observations!), directly taken from the [official Spotify Charts website](https://spotifycharts.com/regional). This was the only dataset that we could find containing the top songs of several countries on Spotify.<br><br>
While we did begin to work with this Kaggle dataset, after becoming more comfortable with pandas and learning about web scraping, we decided that we could make a dataset from the charts ourselves that better suited our research question. By having access to later years' song data (not only 2017), we would be able to answer our research question with more relevant conclusions and do more statistical analyses. For example, in our final project we train a model using 2019 song data to predict a country's happiness score in 2020.

### II. Happiness Datasets Cleaning<a name="cleaning"></a>

Before we could start calling our scraping functions on countries, we needed to identify the top and bottom 10 countries for 2019 and 2020. Below, we clean the Kaggle datasets that compile the happiness scores for these two years. 

We read in the Kaggle dataset that contains the happiness scores for countries in 2019:

In [64]:
happy2019=pd.read_csv("2019.csv")
happy2019.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


Since we are using countries' happiness scores and a country's top music, we only need to keep the happiness scores for countries that spotify has data on. `allcountries` is a list that contains the names of all the countries that Spotify has data for. 

In [65]:
allcountries=["United States", "United Kingdom", "United Arab Emirates", "Argentina", "Austria", "Australia", "Belgium", "Bulgaria", "Bolivia", "Brazil", "Canada", "Switzerland", "Chile", "Colombia", "Costa Rica", "Cyprus", "Czech Republic", "Germany", "Denmark", "Dominican Republic", "Ecuador", "Estonia", "Egypt", "Spain", "Finland", "France", "Greece", "Guatemala", "Hong Kong", "Honduras", "Hungary", "Indonesia", "Ireland", "Israel", "India", "Iceland", "Italy", "Japan", "Republic of Korea", "Lithuania", "Luxembourg", "Latvia", "Morocco", "Mexico", "Malaysia", "Nicaragua", "Netherlands", "Norway", "New Zealand", "Panama", "Peru", "Philippines", "Poland", "Portugal", "Paraguay", "Romania", "Russia", "Saudi Arabia", "Sweden", "Singapore", "Slovakia", "El Salvador", "Thailand", "Turkey", "Taiwan", "Ukraine", "Uruguay", "Vietnam", "South Africa"
]

We then subsetted `happy2019` to only include the countries that appear in both the Kaggle 2019 happiness data set and the Spotify Charts website. We also cleaned the column titles for ease of use later.

In [66]:
cols2019= [x.lower() for x in happy2019.columns] 
cols2019= [x.replace(" ","_") for x in cols2019] 
happy2019.columns=cols2019
countries2019=happy2019.country_or_region.to_numpy()
new=[]
for x in countries2019:
    if x in allcountries:
        new.append(x)
happy2019=happy2019.loc[happy2019['country_or_region'].isin(new)]
happy2019.reset_index(drop=True)

,overall_rank,country_or_region,score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
...,...,...,...,...,...,...,...,...,...
63,97,Bulgaria,5.011,1.092,1.513,0.815,0.311,0.081,0.004
64,106,South Africa,4.722,0.960,1.351,0.469,0.389,0.130,0.055
65,133,Ukraine,4.332,0.820,1.390,0.739,0.178,0.187,0.010
66,137,Egypt,4.166,0.913,1.039,0.644,0.241,0.076,0.067


We continue to rename columns in the 2019 happiness dataset in order for the 2019 and 2020 datasets to have the same column headings. This helps our data analysis code run on each dataframe

In [67]:
happy2019.rename({"country_or_region":"country"}, axis="columns", inplace=True)
happy2019.rename({"score":"happiness_score"}, axis="columns", inplace=True)
happy2019.rename({"overall_rank":"rank"}, axis="columns", inplace=True)
happy2019=happy2019[["rank","happiness_score","country"]]

Another issue we found with scraping the Spotify Charts website was that even if Spotify has data regarding the top songs that a country listens to, they don't always have that data for every day. This issue was especially prominent when we were trying to scrape the countries with the 10 lowest happiness scores. Spotify did not have data available for the entire year of 2019 for countries such as Ukraine and Egypt. To replace these countries, we replaced this country with the next worst happiness score. 

In [74]:
available=pd.read_csv("happy2019.csv", index_col=0)
available_countries=available.country.to_numpy()
happy2019=happy2019.loc[happy2019['country'].isin(available_countries)]
happy2019=happy2019.reset_index(drop=True)

Below is our cleaned dataset for the top 10 and bottom 10 countries in 2019 in terms of their happiness scores. 

In [75]:
happy2019

,rank,happiness_score,country
0,1,7.769,Finland
1,2,7.600,Denmark
2,3,7.554,Norway
3,4,7.494,Iceland
4,5,7.488,Netherlands
5,6,7.480,Switzerland
6,7,7.343,Sweden
7,8,7.307,New Zealand
8,9,7.278,Canada
9,10,7.246,Austria


Next we go through the same process to create the happiness score dataset for countries in 2020.

In [100]:
happy2020=pd.read_csv("happiness2020.csv")
happy2020['rank']=np.arange(1,len(happy2020)+1)
happy2020.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,rank
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,...,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835,1
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,...,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741,2
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,...,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267,3
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,...,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688,4
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,...,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266,5


In [101]:
cols2020= [x.lower() for x in happy2020.columns] 
cols2020= [x.replace(" ","_") for x in cols2020] 
happy2020.columns=cols2020
countries2020=happy2020.country_name.to_numpy()
new=[]
for x in countries2020:
    if x in allcountries:
        new.append(x)
topbot2020=new[:10]+new[-10:]
happy2020=happy2020.loc[happy2020['country_name'].isin(topbot2020)]
happy2020.rename({"country_name":"country"}, axis="columns", inplace=True)
happy2020.rename({"ladder_score":"happiness_score"}, axis="columns", inplace=True)

happy2020=happy2020[["rank","happiness_score","country"]]

In [102]:
happy2020

,rank,happiness_score,country
0,1,7.8087,Finland
1,2,7.6456,Denmark
2,3,7.5599,Switzerland
3,4,7.5045,Iceland
4,5,7.4880,Norway
5,6,7.4489,Netherlands
6,7,7.3535,Sweden
7,8,7.2996,New Zealand
8,9,7.2942,Austria
9,10,7.2375,Luxembourg


We still had the same issue that Spotify does not provide a full year's worth of data regarding a country's top songs per day, so we replace countries for which  we don't have a complete set of data.

In [105]:
available=pd.read_csv("happy2020.csv", index_col=0)
available_countries=available.country.to_numpy()
happy2020=happy2020.loc[happy2020['country'].isin(available_countries)]
happy2020=happy2020.reset_index(drop=True)

In [106]:
happy2020

,rank,happiness_score,country
0,1,7.8087,Finland
1,2,7.6456,Denmark
2,3,7.5599,Switzerland
3,4,7.5045,Iceland
4,5,7.4880,Norway
5,6,7.4489,Netherlands
6,7,7.3535,Sweden
7,8,7.2996,New Zealand
8,9,7.2942,Austria
9,82,5.3843,Malaysia


We saved both datasets as CSV files so we could continue to use them in our main analysis.

In [109]:
happy2019.to_csv("happy2019.csv")

In [107]:
happy2020.to_csv("happy2020.csv")

### III. Web Scraping <a name="scraping"></a>

On a page on https://spotifycharts.com , Spotify allows users to select a date and a country to see a table of the "Top 200", a ranking of the top 200 songs by number of streams (not "Viral 50", which ranks songs by user-sharing). The following functions will create a pandas DataFrame by scraping the table from this webpage. Below is a screenshot of the Spotify Charts website that shows the top 10 songs across the world on April 26th, 2021. 

<img src="spotify_chart.png" width="1000">

In [1]:
#relevant imports
import requests #package for http requests
import bs4 #package for html parsing
import numpy as np
import pandas as pd 
from datetime import timedelta, date
import cloudscraper #for bypassing Spotify cloudflare protection. explained in section 2.5

In order to access the top songs for a certain country on the Spotify Charts website, a user has to select the country from the drop-down menu, which loads a new page. The day that the user is looking at also loads a new page. Therefore, looking at a certain country's top songs for an entire year would generate around 365 links. Since we looked at 20 countries for both 2019 and 2020, we needed 365 links for 40 countries. Defining functions that created and saved these links helped simplify this process. After looping through each link for a country, we were able to get the information listed on each page that a link was connected to.

We created the three functions below, `create_links_2019`, `create_links_2020`, and `daterange`, to get the links for each country more effectively. `daterange(start_date, end_date)` acts as a helper method to  `create_links_2019` and `create_links_2020`.

When`create_links_2019` is called on a country, it returns a list of links. These links are connected to the Spotify Charts website that has the top 200 songs for a certain day. The list only contains links to the top 200 songs in a country between January 1st, 2019 and December 31st, 2019. This method was necessary since we created separate datasets for Spotify data from 2019 and Spotify data from 2020.

In [9]:
# It creates the list of links in 2019 we will web scrape from.
def create_links_2019(country):
    '''
    returns: an array of strings, which are 365 links to spotify charts for each day in 2020 for a specified country.
    parameter: a string. a country's ISO Alpha-2 country abbreviation. These are 2-letter internationally recognized codes.
    https://www.nationsonline.org/oneworld/country_code_list.html for a full list of ISO Alpha-2 country codes.
    eg. to get 365 links for the united states, call create_links_2019('us')
    '''
    start_date = date(2019, 1, 1)
    end_date = date(2019,12,31)
    links = []
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links)

When`create_links_2020` is called on a country, it does the exact same thing as `create_links_2019` except that the returned list only contains links to the top 200 songs in a country between January 1st, 2020 and December 31st, 2020. This method was called only when we were creating the Spotify dataset that only contained information regarding what music countries in 2020 listened to.

In [10]:
def create_links_2020(country):
    '''
    returns: an array of strings, which are 364 links to spotify charts for each day in 2019 for a specified country.
    parameter: a string. a country's ISO Alpha-2 country abbreviation. These are 2-letter internationally recognized codes.
    https://www.nationsonline.org/oneworld/country_code_list.html for a full list of ISO Alpha-2 country codes.
    eg. to get 365 links for the united states, call create_links_2020('us')
    '''
    start_date = date(2020, 1, 1)
    end_date = date(2020,12,31)
    links = []
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links)

`daterange(start_date, end_date)` returns a daterange object that is looped through in either `create_links_2019` or `create_links_2020` in order to append a link for every day of the year.

In [11]:
def daterange(start_date, end_date):
    '''
    helper function for create_links_2019 and create_links_2020
    returns: daterange object.
    parameters: a start date and end date in "%Y-%m-%d" format.
    '''
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

Below, we call `create_links_2019` on the United States to demonstrate:

In [12]:
links_us=create_links_2019('us')

start_date = date(2019, 1, 1)
end_date = date(2019,12,31)
index=0
for single_date in daterange(start_date, end_date):
        print('Link to top 200 songs on ' + single_date.strftime("%Y-%m-%d")+': '+links_us[index])
        index=index+1

Link to top 200 songs on 2019-01-01: https://spotifycharts.com/regional/us/daily/2019-01-01
Link to top 200 songs on 2019-01-02: https://spotifycharts.com/regional/us/daily/2019-01-02
Link to top 200 songs on 2019-01-03: https://spotifycharts.com/regional/us/daily/2019-01-03
Link to top 200 songs on 2019-01-04: https://spotifycharts.com/regional/us/daily/2019-01-04
Link to top 200 songs on 2019-01-05: https://spotifycharts.com/regional/us/daily/2019-01-05
Link to top 200 songs on 2019-01-06: https://spotifycharts.com/regional/us/daily/2019-01-06
Link to top 200 songs on 2019-01-07: https://spotifycharts.com/regional/us/daily/2019-01-07
Link to top 200 songs on 2019-01-08: https://spotifycharts.com/regional/us/daily/2019-01-08
Link to top 200 songs on 2019-01-09: https://spotifycharts.com/regional/us/daily/2019-01-09
Link to top 200 songs on 2019-01-10: https://spotifycharts.com/regional/us/daily/2019-01-10
Link to top 200 songs on 2019-01-11: https://spotifycharts.com/regional/us/daily

Next, we define `get_table(link)` and `track_artist`. These functions read the html of the Spotify Charts website and then add the relevant data to a dataframe.

`get_table(link)` uses the pandas function `read_html()` and Beautiful Soup to return a dataframe of the top 50 songs for whatever Spotify Charts page that the function has been called on. The dataframe that `get_table(link)` returns includes the song's `position` (where it was ranked that day), `track` (the name of the track and who the track's artist), `streams` (the number of times the song has been streamed), `date` (the date that the user is looking at), and `url` (the link to the song on Spotify). 

As you can see, we use a package called `cloudscraper` instead of `requests` in order to get and read the information that the link contains. We intiially were using the `requests` package to perform our webscraping, however, Spotify installed CloudFlare DDOS protection on their website mid-project which required us to switch the packages we used. 

In [21]:
def get_table(link):
    '''
    a function that returns a table of the top 50 songs from a spotify chart
    with the following columns:
    track. a string, "[track name] by [artist]"
    position. an int, the ranking of the track on the chart.
    streams. an int, # of streams of the track on a specified date.
    date. a date time obj. the day the song was streamed.
    url. a url to the song on Spotify.
    
    argument: a string, which is a link to a Spotify chart. eg. 'https://spotifycharts.com/regional/us/daily/2020-01-01'
    '''
    #getting the df
    scraper = cloudscraper.create_scraper()
    r = scraper.get(link)
    df_list = pd.read_html(r.text) # this parses all the tables in webpages to a lis
    df = df_list[0]
    #for the purposes of our project, we only want the top 50 songs of the table that was scraped from the
    #spotify website, which gives us top 200. we find that 200 songs per day was excessive
    df=df.head(50)

    #cleaning column names and dropping irrelevant columns
    #to lowercase
    cols= [x.lower() for x in df.columns] 
    df.columns=cols
    #after scraping, Spotify returns a column 'unnamed: 1', which is actually the position of the song on the chart.
    #which we have renamed
    df = df.rename(columns={'unnamed: 1': 'position'})
    #it also returns two NaN columns, which we drop here
    df=df.drop(columns=["unnamed: 0","unnamed: 2"])
    
    #adding a date column by parsing the link used in the argument.
    df["date"]=link[-10:]
    df['date']=pd.to_datetime(df['date'])
    
    #creating the url column
    #using Beautiful soup to webscrape the links on the site.
    datasoup = bs4.BeautifulSoup(r.text, 'html.parser')
    aList=[]
    #find all 'a' tags
    for data in datasoup.findAll('a'):
        aList.append(data)
    links=[]
    #find all href tags
    for x in aList:
        links.append(x.get('href'))
    #drop None types
    clean = filter(None, links)
    urls=[]
    #only keep the links that are links to spotify tracks.
    for x in clean:
        if 'open.spotify.com/track/' in x:
            urls.append(x)
    #only append the top 50 to the dataframe
    df['url']=urls[:50]
    return df

Before using `cloudscraper`, when we ran `get_table(link)` we received the error message:

In [20]:
test=get_table('https://spotifycharts.com/regional/us/daily/2021-05-09')

ValueError: No tables found

Once using `cloudscraper`, this is the dataframe that `get_table(link)` returns when we call it on the link to the top 200 songs in the United States on May 5th, 2021:

In [22]:
get_table('https://spotifycharts.com/regional/us/daily/2021-05-09')

,position,track,streams,date,url
0,1,Kiss Me More (feat. SZA) by Doja Cat,1490138,2021-05-09,https://open.spotify.com/track/748mdHapucXQri7...
1,2,RAPSTAR by Polo G,1369211,2021-05-09,https://open.spotify.com/track/43PGPuHIlVOc04j...
2,3,Miss The Rage (feat. Playboi Carti) by Trippi...,1159962,2021-05-09,https://open.spotify.com/track/2BITQ360Knh6qNA...
3,4,i n t e r l u d e by J. Cole,1090993,2021-05-09,https://open.spotify.com/track/2gyYjQWXKxemhAy...
4,5,MONTERO (Call Me By Your Name) by Lil Nas X,1077197,2021-05-09,https://open.spotify.com/track/67BtfxlNbhBmCDR...
5,6,Levitating (feat. DaBaby) by Dua Lipa,1016614,2021-05-09,https://open.spotify.com/track/5nujrmhLynf4yMo...
6,7,Peaches (feat. Daniel Caesar & Giveon) by Jus...,997462,2021-05-09,https://open.spotify.com/track/4iJyoBOLtHqaGxP...
7,8,deja vu by Olivia Rodrigo,823109,2021-05-09,https://open.spotify.com/track/61KpQadow081I2A...
8,9,Astronaut In The Ocean by Masked Wolf,742175,2021-05-09,https://open.spotify.com/track/3VT8hOC5vuDXBsH...
9,10,"Leave The Door Open by Bruno Mars, Anderson ....",698458,2021-05-09,https://open.spotify.com/track/7MAibcTli4IisCt...


As you can see in the dataframe above, `get_table(link)` puts a track name and the track's artist in the same column. We want track name and track artist to be in two separate columns. This is what `track_artist(df)` does. 

In [23]:
def track_artist(df):
    '''
    returns a df that takes "track" column, which contains both the artist and song. 
    it is a string formatted as "[track name] by [artist]"
    and parses it into 2 columns: track_name and artist.
    and deletes the track column after.
    
    argument: a dataframe created by get_table(), containing a column "track"
    with each observation a string formatted as "[track name] by [artist]"
    '''
    tracks=df.track.values
    songtitles=[]
    songartists=[]
    for song in tracks:
        index1=song.find("  by")
        index2=song.find("by ")
        track=song[:index1]
        songtitles.append(track)
        artist=song[index2+3:]
        songartists.append(artist)
    df["track_name"]=songtitles
    df["artist"]=songartists
    df=df.drop(columns=["track"])
    return df

Once we defined all of our functions, it was time to actually call the functions on each country. Instead of using a for-loop to call `get_table(link)` and `track_artist(df)` on all of the countries in `happy2019` and `happy2020`, we manually ran all of these functions on the 40 countries. We did this in order to run our functions on multiple countries at once, using different kernels. We also print the links since sometimes the webscraper would crash at a random date. Printing the links would let us see where the webscraper stopped, and then modify our the dates for which we were webscraping. 

Shown below is an example of how we ran our function on Norway. We did this for every country, replacing "Norway" with the country name and "no" with the 2-letter abbreviation for the country.

In [10]:
norway=create_links_2019("no")
norway2019=pd.DataFrame()

In [11]:
for link in norway:
    df=get_table(link)
    df=track_artist(df)
    df['region']="no"
    norway2019=norway2019.append(df,ignore_index=True)
    print(link)
norway2019['region']='Norway'
norway2019

https://spotifycharts.com/regional/no/daily/2019-01-01
https://spotifycharts.com/regional/no/daily/2019-01-02
https://spotifycharts.com/regional/no/daily/2019-01-03
https://spotifycharts.com/regional/no/daily/2019-01-04
https://spotifycharts.com/regional/no/daily/2019-01-05
https://spotifycharts.com/regional/no/daily/2019-01-06
https://spotifycharts.com/regional/no/daily/2019-01-07
https://spotifycharts.com/regional/no/daily/2019-01-08
https://spotifycharts.com/regional/no/daily/2019-01-09
https://spotifycharts.com/regional/no/daily/2019-01-10
https://spotifycharts.com/regional/no/daily/2019-01-11
https://spotifycharts.com/regional/no/daily/2019-01-12
https://spotifycharts.com/regional/no/daily/2019-01-13
https://spotifycharts.com/regional/no/daily/2019-01-14
https://spotifycharts.com/regional/no/daily/2019-01-15
https://spotifycharts.com/regional/no/daily/2019-01-16
https://spotifycharts.com/regional/no/daily/2019-01-17
https://spotifycharts.com/regional/no/daily/2019-01-18
https://sp

,position,streams,date,url,track_name,artist,region
0,1,69663,2019-01-01,https://open.spotify.com/track/25sgk305KZfyuqV...,Sweet but Psycho,Ava Max,no
1,2,67916,2019-01-01,https://open.spotify.com/track/6MWtB6iiXyIwun0...,Wow.,Post Malone,no
2,3,56131,2019-01-01,https://open.spotify.com/track/0lG6QM1jjbbX5UU...,Lost Control,"Alan Walker, Sorana",no
3,4,53922,2019-01-01,https://open.spotify.com/track/2VxeLyX666F8uXC...,Shallow,"Lady Gaga, Bradley Cooper",no
4,5,53118,2019-01-01,https://open.spotify.com/track/0nrWZWkJM62ftQJ...,Diamond Heart,"Alan Walker, Sophia Somajo",no
...,...,...,...,...,...,...,...
18195,46,25397,2019-12-30,https://open.spotify.com/track/285pBltuF7vW8Te...,Lucid Dreams,Juice WRLD,no
18196,47,25119,2019-12-30,https://open.spotify.com/track/4AO1XhrgJczQ9bN...,WHAT TO DO? (feat. Don Toliver),"JACKBOYS, Travis Scott",no
18197,48,23253,2019-12-30,https://open.spotify.com/track/4QPEwDdOMhY7rNS...,Own It (feat. Ed Sheeran & Burna Boy),Stormzy,no
18198,49,22865,2019-12-30,https://open.spotify.com/track/6TmkOBZ2UEQaIWR...,Can I Be Forgiven,Highasakite,no


We then saved each dataframe (which contained the daily top 50 songs that a country listened to for a year) as its own csv file. Below is again an example using Norway. 

In [28]:
norway2019.to_csv('norway2020.csv')

Once all of the 20 countries had been put through our webscraper, we concatenated all of the csv files together. We did this for the top and bottom 10 happiest countries in 2020 to create the dataset `spotify2020`. We did this for the top and bottom 10 happiest countries in 2019 to create the dataset `spotify2019`.

Shown below is how we created `spotify2020`. This code was run after we read in the country csv files that we created using the method shown above. 

In [ ]:
spotify2020=pd.DataFrame()
spotify2020=spotify2020.append(finland,ignore_index=True)
spotify2020=spotify2020.append(denmark,ignore_index=True)
spotify2020=spotify2020.append(switzerland,ignore_index=True)
spotify2020=spotify2020.append(iceland,ignore_index=True)
spotify2020=spotify2020.append(norway,ignore_index=True)
spotify2020=spotify2020.append(netherlands,ignore_index=True)
spotify2020=spotify2020.append(sweden,ignore_index=True)
spotify2020=spotify2020.append(newzealand,ignore_index=True)
spotify2020=spotify2020.append(austria,ignore_index=True)
spotify2020=spotify2020.append(canada,ignore_index=True)

After webscraping, we are left with a dataset for 2019 and a dataset for 2020 that contains the daily top 50 songs per country for the top and bottom 10 countries in regards to their happiness scores! Shown below to demonstrate is `spotify2020`.

In [32]:
spotify2020

,position,streams,date,url,track_name,artist,region
0,1,39681,2020-01-01,https://open.spotify.com/track/0tc8HGXosQDC8TT...,Hei rakas,BEHM,fi
1,2,31179,2020-01-01,https://open.spotify.com/track/3eHkFrUUGYuwEgI...,Pintakaasulla,JVG,fi
2,3,30339,2020-01-01,https://open.spotify.com/track/4gEvvWYAF3yzv9h...,Luota Muhun,ibe,fi
3,4,30208,2020-01-01,https://open.spotify.com/track/0sf12qNH5qcw8qp...,Blinding Lights,The Weeknd,fi
4,5,28966,2020-01-01,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,Dance Monkey,Tones And I,fi
...,...,...,...,...,...,...,...
182445,46,58237,2020-12-30,https://open.spotify.com/track/6f3Slt0GbA2bPZl...,The Business,Tiësto,ca
182446,47,57657,2020-12-30,https://open.spotify.com/track/7kDUspsoYfLkWnZ...,my ex's best friend (with blackbear),Machine Gun Kelly,ca
182447,48,57321,2020-12-30,https://open.spotify.com/track/21jGcNKet2qwijl...,Circles,Post Malone,ca
182448,49,57003,2020-12-30,https://open.spotify.com/track/0A1hoCfMLkiAgvh...,Body,Megan Thee Stallion,ca


### IV. Spotify API <a name="api"></a>

We used a package called `Spotipy` which aided us in our use of Spotify API. We import `spotipy` and other necessary packages below.

In [111]:
#Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from sklearn.linear_model import LinearRegression as lr
# Deleted Spotify Web Developer CID and Secret

Next we developed a function, `getunique(dataframe)` that would allow us to get the features of the daily top 50 songs that a country listens to. The API provides a score that provides a score for a song's danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, and tempo. 

We call `drop_duplicates()` on the dataframe in order to get around the limit to the number of calls we can make to the Spotify API; this makes sense to use since the top songs are not wildly different from day-to-day. We can only call `getunique(dataframe)` on unique songs and then merge the song feature information to all of the songs later.   

In [112]:
def getunique(dataframe):
    '''
    creates a dataframe containing the audio features of songs, all scores, which are floats. These audio features are provided by the Spotify API.
    parameters: a dataframe with a 'url' column, which contains valid links to a song on Spotify.
    returns: a dataframe with the audio features as columns, the observations being
    ONLY for the unique songs in the provided dataframe. Because the dataframe contains several repeated songs,
    we will avoid calling the API on duplicated songs because we will get a "max entries error"
    '''
    audiofeatures=pd.DataFrame({'danceability':[], 'energy':[],'key':[], 'loudness':[], 'mode':[], 'speechiness':[],'acousticness':[], 'instrumentalness':[],'liveness':[], 'valence':[], 'tempo':[], 'type':[],'id':[], 'uri':[],'track_href':[], 'analysis_url':[], 'duration_ms':[], 'time_signature':[]})
    dataframe.drop_duplicates(subset ="track_name",
                     keep = 'first', inplace = True,) #drop duplicate songs
    dataframe=dataframe.reset_index(drop=True)

    for row in range(len(dataframe)):
            url=dataframe.loc[row,'url']
            sub=url.rindex('/')
            idurl=url[sub+1:]
            newrow=sp.audio_features(idurl)[0]
            audiofeatures=audiofeatures.append(newrow,ignore_index=True)
    audiofeatures['track_name']=dataframe['track_name']
    audiofeatures['happiness_score']=dataframe['happiness_score']
    audiofeatures['happiness_rank']=dataframe['happiness_rank']
    audiofeatures['country']=dataframe['country']
    return audiofeatures

Below we demonstrate the use of `getunique(dataframe)` on data from 2019. We pass `spotify2019` as the argument. We do the same for 2020, instead using the relevant dataframes for 2020 not 2019. 

In [181]:
uniquesongs2019=getunique(spotify2019)

In [ ]:
uniquesongs2020=getunique(spotify2020)

Next, we merge our the data we cleaned earlier regarding the happiness scores for the top and bottom countries in 2019. We merge this data to the dataframe on `country` as this information appears in both the `uniquesongs2019` and the `happy2019` dataset. We do the same for 2020.

In [307]:
uniquesongs2019=uniquesongs2019.merge(happy2019[['happiness_rank','country','happiness_score']], how="left",on='country')

In [ ]:
uniquesongs2020=uniquesongs2019.merge(happy2020[['happiness_rank','country','happiness_score']], how="left",on='country')

Finally, we merge our dataframe that only contains unique songs, with the original dataframe. This leaves us with the dataframes that we use in our main analysis. 

In [310]:
spotify2019=uniquesongs2019.merge(song[['danceability','track_name' ,'happiness_score', 'happiness_rank', 'energy', 'key', 'loudness', 'mode', 'speechiness',
'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms']], how="left",on='track_name')

In [ ]:
spotify2020=uniquesongs2020.merge(song[['danceability','track_name' ,'happiness_score', 'happiness_rank', 'energy', 'key', 'loudness', 'mode', 'speechiness',
'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms']], how="left",on='track_name')

Shown below is our dataframe for 2019 that contains the happiness scores and the song feature information.

In [311]:
spotify2019

,position,streams,date,url,track_name,artist,region,country,danceability,happiness_score,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,1,33717,2019-01-01,https://open.spotify.com/track/6MWtB6iiXyIwun0...,Wow.,Post Malone,fi,Finland,0.833,7.769,...,-7.399,0.0,0.1780,0.16300,0.000002,0.1010,0.385,99.947,6MWtB6iiXyIwun0YzU6DFP,149520.0
1,2,29651,2019-01-01,https://open.spotify.com/track/25sgk305KZfyuqV...,Sweet but Psycho,Ava Max,fi,Finland,0.719,7.769,...,-4.724,1.0,0.0476,0.06910,0.000000,0.1660,0.628,133.002,25sgk305KZfyuqVBQIahim,187436.0
2,3,28329,2019-01-01,https://open.spotify.com/track/4RYtaqxjDJUOY2G...,Harmaa Rinne,Teflon Brothers,fi,Finland,0.861,7.769,...,-5.862,0.0,0.1650,0.04100,0.000000,0.1500,0.962,129.933,4RYtaqxjDJUOY2GrtkLTFf,170886.0
3,4,23977,2019-01-01,https://open.spotify.com/track/2rPE9A1vEgShuZx...,"thank u, next",Ariana Grande,fi,Finland,0.724,7.769,...,-5.642,1.0,0.0658,0.28000,0.000000,0.1020,0.435,106.960,2rPE9A1vEgShuZxxzR2tZH,207333.0
4,5,22435,2019-01-01,https://open.spotify.com/track/00WO1oBxZcj9aBo...,Tavallinen,Keko Salata,fi,Finland,0.803,7.769,...,-7.928,0.0,0.0564,0.67300,0.000010,0.1330,0.404,112.964,00WO1oBxZcj9aBoeiODXDx,231791.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363995,46,3427,2019-12-30,https://open.spotify.com/track/3jjujdWJ72nww5e...,Adore You,Harry Styles,za,South Africa,0.676,7.769,...,-3.675,1.0,0.0483,0.02370,0.000007,0.1020,0.569,99.048,1M4qEo4HE3PRaCOM7EXNJq,207133.0
363996,47,3416,2019-12-30,https://open.spotify.com/track/72Yg5qdIqpTnXrN...,Dames,Biggy,za,South Africa,0.912,4.722,...,-7.984,1.0,0.2260,0.10300,0.000006,0.3580,0.637,122.024,72Yg5qdIqpTnXrNPEewglz,217879.0
363997,48,3413,2019-12-30,https://open.spotify.com/track/6XHVuErjQ4XNm6n...,No Guidance (feat. Drake),Chris Brown,za,South Africa,0.698,7.600,...,-6.756,0.0,0.1510,0.12300,0.000000,0.1550,0.137,92.932,7FEwp8BavoEVE3AnxJDchc,260640.0
363998,49,3407,2019-12-30,https://open.spotify.com/track/7h0d2h0fUmzbs7z...,HIGHEST IN THE ROOM (feat. ROSALÍA & Lil Baby)...,Travis Scott,za,South Africa,0.559,7.769,...,-6.406,0.0,0.0346,0.05670,0.000000,0.0992,0.114,153.131,7h0d2h0fUmzbs7zeFigJPn,244874.0


In [341]:
spotify2019.to_csv('spotify2019.csv')

In [ ]:
spotify2020.to_csv('spotify2020.csv')